In [41]:
import polars as pl
import json
import os

In [42]:
EXCEL_PATH = rf"Transporte (1).xlsx"

import polars as pl

df_raw = pl.read_excel(
    source=EXCEL_PATH,
    engine="calamine",
    table_name="Tabla1"
)


new_name_columns = {
    "SPEC405   [#]   Trips": "n_envios",
    "SPEC402   [#]   Drops": "n_entregas",
    "SPECT1     [#]   Transport days": "dias_transporte",
    "SPEC407   [KM]   Paid KM": "kilometros_recorridos",
    "SPEC801   [#]   Base Pallets Units": "n_pallets",
    "SPEC409   [EUR]   Total Trip Cost": "coste"
}


df = df_raw.lazy().with_columns(
    
    # 1. hallar el pais de destino del envío
    pl.col("Planta").str.replace("SK ", ""),
    pl.when(pl.col("Destino").str.contains("To-ES"))
    .then(pl.lit("España"))
    .when(pl.col("Destino").str.contains("To-PT"))
    .then(pl.lit("Portugal"))
    .otherwise("Destino").alias("pais_destino"),
    
    # 2. Sacamos el Código postal
    pl.col("Destino").str.extract(r"(\d{4}-\d{3}|\d{5})") 
    .alias("codigo_postal"),

    # 3. Ahora el Municipio
    pl.col("Destino").str.strip_chars()
    .str.extract(r"^\S+\s+\S+\s+(.*)") 
    .str.to_titlecase()
    .alias("municipio_destino"),
    
    # 4. Formamos la fecha
    pl.col("Fecha").str.to_date(format="%B %Y")
    .alias("Fecha")
    ).filter(
        pl.col("tipo_envio").str.strip_chars() != "TM Non-Corrugated"
    ).rename(new_name_columns)

df_filtered = df.collect().filter(

    # Filtramos por Agosto del 2025
    pl.col("Fecha").dt.year() == 2025,
    pl.col("Fecha").dt.month() == 8
    )

df_filtered

Planta,Destino,Tranportista,Fecha,tipo_envio,n_envios,n_entregas,dias_transporte,kilometros_recorridos,n_pallets,coste,pais_destino,codigo_postal,municipio_destino
str,str,str,date,str,f64,f64,i64,f64,i64,f64,str,str,str
"""Celpack""","""To-ES 19200 AZUQUECA DE HENARE…","""TC CARRERAS GRUPO LOGISTICO, S…",2025-08-01,"""TM Third-party_FTL/CTL""",3.33334,4.0,4,1725.33944,186,1986.53625,"""España""","""19200""","""Azuqueca De Henares"""
"""Celpack""","""To-ES 19200 AZUQUECA DE HENARE…","""TC OPS-TRANSPORTES UNIPESSOAL,…",2025-08-01,"""TM Third-party_FTL/CTL""",1.0,1.0,1,563.525,24,650.0,"""España""","""19200""","""Azuqueca De Henares"""
"""Celpack""","""To-ES 28806 ALCALA DE HENARES""","""TC CARRERAS GRUPO LOGISTICO, S…",2025-08-01,"""TM Third-party_FTL/CTL""",0.33333,1.0,1,436.8405,30,473.10928,"""España""","""28806""","""Alcala De Henares"""
"""Celpack""","""To-ES 28806 ALCALA DE HENARES""","""TC OPS-TRANSPORTES UNIPESSOAL,…",2025-08-01,"""TM Third-party_FTL/CTL""",1.0,1.0,1,580.715,56,650.0,"""España""","""28806""","""Alcala De Henares"""
"""Celpack""","""To-ES 28320 PINTO""","""TC CARRERAS GRUPO LOGISTICO, S…",2025-08-01,"""TM Third-party_FTL/CTL""",3.33333,4.0,4,1820.82307,93,2090.35447,"""España""","""28320""","""Pinto"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Vigo""","""To-PT 4905-220 VIANA DO CASTEL…","""TC TT ALBERTO GONZALEZ PARENTE""",2025-08-01,"""TM Third-party_FTL/CTL""",0.5,1.0,1,72.07723,2,53.60305,"""Portugal""","""4905-220""","""Viana Do Castelo"""
"""Vigo""","""To-PT 4920-247 VILA NOVA DE CE…","""TC GRANXATRANS LOGISTICA,S.L.""",2025-08-01,"""TM Third-party_FTL/CTL""",5.97343,12.28024,7,209.43699,177,1092.75298,"""Portugal""","""4920-247""","""Vila Nova De Cerveira"""
"""Vigo""","""To-PT 4920-247 VILA NOVA DE CE…","""TC TT ALBERTO GONZALEZ PARENTE""",2025-08-01,"""TM Third-party_FTL/CTL""",3.0,6.0,5,196.01712,44,208.60071,"""Portugal""","""4920-247""","""Vila Nova De Cerveira"""


In [43]:
def cargar_referencia_postal(path_archivo):
    # Definimos los nombres según el readme de GeoNames
    columnas = [
        "country_code", "postal_code", "place_name", 
        "admin_name1", "admin_code1", "admin_name2", "admin_code2", 
        "admin_name3", "admin_code3", "latitude", "longitude", "accuracy"
    ]
    
    return pl.read_csv(
        path_archivo,
        separator="\t",        # El readme dice "tab-delimited"
        has_header=False,      # El readme dice que no tiene cabecera
        new_columns=columnas,
        encoding="utf8",       # Codificación estándar
        infer_schema_length=0, # Leemos todo como string primero para no fallar
    ).with_columns([
        pl.col("latitude").cast(pl.Float64),
        pl.col("longitude").cast(pl.Float64)
    ])
    
    
def fusionar_coordenadas_robustas(df_logistica, df_referencia, col_cp_log="CP"):
    """
    Une datos de logística con coordenadas de GeoNames de forma robusta.
    Maneja diferencias de tipos (int vs str) y espacios en blanco.
    """
    
    # 1. Normalizar el DataFrame de Referencia (GeoNames)
    # Forzamos postal_code a String, quitamos espacios y aseguramos formato
    df_ref_clean = df_referencia.select([
        pl.col("postal_code").cast(pl.String).str.strip_chars().alias("cp_ref"),
        pl.col("latitude"),
        pl.col("longitude")
    ])

    # 2. Normalizar el DataFrame de Logística
    # Convertimos a String y manejamos posibles nulos o formatos numéricos
    df_log_clean = df_logistica.with_columns(
        pl.col(col_cp_log).cast(pl.String).str.strip_chars().alias("cp_join")
    )

    # 3. Realizar el JOIN
    # Usamos las columnas normalizadas para asegurar la coincidencia
    resultado = df_log_clean.join(
        df_ref_clean,
        left_on="cp_join",
        right_on="cp_ref",
        how="left"
    ).drop("cp_join") # Borramos la columna auxiliar de unión

    return resultado

In [44]:
# Cargamos España CP

ES_CP = cargar_referencia_postal(rf"ES.txt")
PT_CP = cargar_referencia_postal(rf"PT.txt")

IBERIA_CP = pl.concat([ES_CP, PT_CP])

IBERIA_CP.head()

country_code,postal_code,place_name,admin_name1,admin_code1,admin_name2,admin_code2,admin_name3,admin_code3,latitude,longitude,accuracy
str,str,str,str,str,str,str,str,str,f64,f64,str
"""ES""","""04001""","""Almeria""","""Andalucia""","""AN""","""Almería""","""AL""","""Almería""","""04013""",36.8381,-2.4597,"""4"""
"""ES""","""04002""","""Almeria""","""Andalucia""","""AN""","""Almería""","""AL""","""Almería""","""04013""",36.8381,-2.4597,"""4"""
"""ES""","""04002""","""El Palmer""","""Andalucia""","""AN""","""Almería""","""AL""","""Almería""","""04013""",36.8381,-2.4597,"""3"""
"""ES""","""04003""","""Almeria""","""Andalucia""","""AN""","""Almería""","""AL""","""Almería""","""04013""",36.8381,-2.4597,"""4"""
"""ES""","""04004""","""Almeria""","""Andalucia""","""AN""","""Almería""","""AL""","""Almería""","""04013""",36.8381,-2.4597,"""4"""


In [45]:

df_final = fusionar_coordenadas_robustas(
    df_filtered,
    IBERIA_CP,
    col_cp_log = "codigo_postal"
    )

df_final = df_final.filter(pl.col("latitude").is_not_null())

In [47]:
	JSON_UBI = df_final.select(
    "municipio_destino", 
    "pais_destino",
    "codigo_postal",
    "latitude", 
    "longitude"
).unique(maintain_order=True)

JSON_UBI_2 = JSON_UBI.group_by("codigo_postal").agg(
        pl.struct([
            "municipio_destino",
            "pais_destino",
            "latitude",
            "longitude"
        ]).alias("datos")
    )

resultado_final = {
    row["codigo_postal"]: row["datos"] 
    for row in JSON_UBI_2.to_dicts()
}

# # 3. Guardamos el archivo JSON
UBI = RF"C:\Users\kike\.gemini\antigravity\scratch\logistics_optimizer\data"
with open(os.path.join(UBI,"cliente_ubi.json"), "w", encoding="utf-8") as f:
    json.dump(resultado_final, f, indent=2, ensure_ascii=False)

print("¡JSON generado con éxito!")

¡JSON generado con éxito!
